**Collect and load dataset**

In [9]:
! pip install kaggle #install Kaggle library

In [10]:
! mkdir ~/.kaggle #make directory named ".kaggle"
! cp kaggle.json ~/.kaggle/ #Copy the “kaggle.json” into this new directory
! chmod 600 ~/.kaggle/kaggle.json #Allocate the required permission for this file.

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [11]:
! kaggle datasets download alessiocorrado99/animals10 #download dataset

 99% 582M/586M [00:05<00:00, 60.5MB/s]
100% 586M/586M [00:05<00:00, 111MB/s] 


In [12]:
! ls /content/raw-img

ls: cannot access '/content/raw-img': No such file or directory


In [13]:
! unzip animals10.zip #unzip file

Streaming output truncated to the last 5000 lines.
  inflating: raw-img/ragno/OIP-MVQChNQpHiMZEVMPehrvoAHaE6.jpeg  
  inflating: raw-img/ragno/OIP-MVdIFDAFpNyK_XCpwkDIWgHaFj.jpeg  
  inflating: raw-img/ragno/OIP-MYY_8k5AHqMJwTCNXfH6rwHaFj.jpeg  
  inflating: raw-img/ragno/OIP-MZEVXuoMsATG8u_UuTQ6hAHaE8.jpeg  
  inflating: raw-img/ragno/OIP-MZg0tznDRE95AMfyuutuYAHaE6.jpeg  
  inflating: raw-img/ragno/OIP-M_r9DkR2X5gP0P5-z8wPYQHaE7.jpeg  
  inflating: raw-img/ragno/OIP-M_xDUSzjlF3-w9XHCm93cgHaFF.jpeg  
  inflating: raw-img/ragno/OIP-MbI4-bMyutRX3X382IAVcQAAAA.jpeg  
  inflating: raw-img/ragno/OIP-MbMksZdD7gomsC9fd6cpKgHaHJ.jpeg  
  inflating: raw-img/ragno/OIP-Md4AMcvjaHcivJqeW06wEwHaFW.jpeg  
  inflating: raw-img/ragno/OIP-MgCgailkT5XkO1XczzQCngHaLK.jpeg  
  inflating: raw-img/ragno/OIP-MgrKq05YHGXsgj1AmYIyJQHaE8.jpeg  
  inflating: raw-img/ragno/OIP-MhRGx7d8nbnHBuqrsO00EQHaFo.jpeg  
  inflating: raw-img/ragno/OIP-MiVgMXcKGQdog2KGe7i0YQHaHh.jpeg  
  inflating: raw-img/ragno/OIP-Mis0lJf3

Importing libraries

In [5]:
import tensorflow as tf
from tensorflow import keras
import sklearn
import numpy as np
np.random.seed(42)
import matplotlib as mpl
import matplotlib.pyplot as plt

# my imports
import os
import cv2
from sklearn.model_selection import train_test_split
from random import sample, seed
import pandas as pd
from tqdm import tqdm
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import (
    Dense, Dropout, Activation, Flatten, MaxPooling2D, Conv2D, ZeroPadding2D, BatchNormalization
)
from keras.regularizers import l2
import gc
import random
from keras.callbacks import CSVLogger

import matplotlib.pyplot as plt

Translate.py (given by author)

> Python list containing classes initiated with variable "translate"

> Used to reference class names of image during visualization stage







In [14]:
translate = {"cane": "dog", "cavallo": "horse", "elefante": "elephant", 
             "farfalla": "butterfly", "gallina": "chicken", "gatto": "cat", 
             "mucca": "cow", "pecora": "sheep", "scoiattolo": "squirrel", 
             "dog": "cane", "cavallo": "horse", "elephant" : "elefante", 
             "butterfly": "farfalla", "chicken": "gallina", "cat": "gatto", 
             "cow": "mucca", "spider": "ragno", "squirrel": "scoiattolo"}

**Model Training and Implementation Code**

In [15]:
keys = translate.keys()
key_index = dict(list(zip(keys, range(len(keys)))))
img_size = 128

In [16]:
random.seed(1)

#find file size

#create empty list for files
files = []

for dirname, _, filenames in os.walk('/content/raw-img'):
    for filename in filenames:
        dirname_arr = dirname.split("/")
        if(random.random() < 0.2):
            files.append(os.path.join(dirname, filename) +','+dirname_arr[3]+',test') #index =3 because otherwise list index out of range
        else:
            files.append(os.path.join(dirname, filename)+','+dirname_arr[3]+',train') #index =3 because otherwise list index out of range
            
files = np.array(files)
print('Number of images/labels loaded: ', files.size)

Number of images/labels loaded:  26179


In [17]:
def get_image_data(file):
    img = image.load_img(file, target_size=(100, 100, 3))

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    return x

Split training and testing data

In [18]:
x_train = []
y_train = []
x_test = []
y_test = []
animals = ('gatto', 'elefante', 'cane', 'scoiattolo', 'mucca', 'farfalla', 'gallina', 'cavallo', 'ragno', 'pecora')

for i in range(1, files.size):
    file_path, animal, phase  = files[i].split(',')
    
    animal_types = keras.utils.to_categorical(animals.index(animal), 10)
    image_data = np.array(get_image_data(file_path))
    
    if phase == 'train':
        x_train.append(image_data)
        y_train.append(animal_types)
    elif phase == 'test':
        x_test.append(image_data)
        y_test.append(animal_types)
    print('animal is '+animal+' as '+phase+' data set '+str(i))
        
        
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')

x_train /= 255
x_test /= 255

x_train = x_train.reshape(x_train.shape[0], 100, 100, 3)
x_train = x_train.astype('float32')
x_test = x_test.reshape(x_test.shape[0], 100, 100, 3)
x_test = x_test.astype('float32')

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

Streaming output truncated to the last 5000 lines.
animal is farfalla as test data set 21181
animal is farfalla as train data set 21182
animal is farfalla as test data set 21183
animal is farfalla as train data set 21184
animal is farfalla as train data set 21185
animal is farfalla as train data set 21186
animal is farfalla as train data set 21187
animal is farfalla as train data set 21188
animal is farfalla as train data set 21189
animal is farfalla as train data set 21190
animal is farfalla as train data set 21191
animal is farfalla as train data set 21192
animal is farfalla as train data set 21193
animal is farfalla as train data set 21194
animal is farfalla as train data set 21195
animal is farfalla as test data set 21196
animal is farfalla as train data set 21197
animal is farfalla as train data set 21198
animal is farfalla as train data set 21199
animal is farfalla as test data set 21200
animal is farfalla as test data set 21201
animal is farfalla as train data set 21202
animal i

Creating AlexNet Model and print results

In [19]:
print("\nCreating convolutional neural network (AlexNet)...")
batch_size = 53
epochs = 69
gc.collect()
alexnet = Sequential()

# Layer 1
alexnet.add(Conv2D(96, (11, 11), input_shape=(100, 100, 3), padding='same', kernel_regularizer=l2(0)))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 2
alexnet.add(Conv2D(256, (5, 5), padding='same'))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 3
alexnet.add(ZeroPadding2D((1, 1)))
alexnet.add(Conv2D(512, (3, 3), padding='same'))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 4
alexnet.add(ZeroPadding2D((1, 1)))
alexnet.add(Conv2D(1024, (3, 3), padding='same'))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))

# Layer 5
alexnet.add(ZeroPadding2D((1, 1)))
alexnet.add(Conv2D(1024, (3, 3), padding='same'))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 6
alexnet.add(Flatten())
alexnet.add(Dense(3072))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(Dropout(0.5))

# Layer 7
alexnet.add(Dense(4096))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(Dropout(0.5))

# Layer 8
alexnet.add(Dense(10))
alexnet.add(BatchNormalization())
alexnet.add(Activation('softmax'))

fit = True 

if fit == True:
    gen = ImageDataGenerator(horizontal_flip=True)
    gen.fit(x_train)
    train_generator = gen.flow(x_train, y_train, batch_size=batch_size)

    alexnet.compile(loss='categorical_crossentropy', 
          optimizer="adam",
          metrics=['accuracy']
    )
    alexnet.fit_generator(train_generator, validation_data=(x_test, y_test), steps_per_epoch=batch_size, epochs=epochs, verbose=1, callbacks=[CSVLogger('training_animal.log', separator=',', append=False)]) 
    alexnet.save('animal.h5')
else:
    alexnet = keras.models.load_model('/kaggle/working/animal.h5') 
    alexnet.history = pd.read_csv('training_animal.log', sep=',', engine='python')


Creating convolutional neural network (AlexNet)...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/69
53/53 [==============================] - 31s 375ms/step - loss: 2.2115 - accuracy: 0.2360 - val_loss: 4.9016 - val_accuracy: 0.1205
Epoch 2/69
53/53 [==============================] - 14s 270ms/step - loss: 1.8616 - accuracy: 0.3528 - val_loss: 3.0363 - val_accuracy: 0.2145
Epoch 3/69
53/53 [==============================] - 14s 271ms/step - loss: 1.7204 - accuracy: 0.4087 - val_loss: 2.3990 - val_accuracy: 0.2472
Epoch 4/69
53/53 [==============================] - 14s 270ms/step - loss: 1.5958 - accuracy: 0.4564 - val_loss: 1.9446 - val_accuracy: 0.3402
Epoch 5/69
53/53 [==============================] - 14s 270ms/step - loss: 1.5386 - accuracy: 0.4817 - val_loss: 3.6854 - val_accuracy: 0.2083
Epoch 6/69
53/53 [==============================] - 14s 269ms/step - loss: 1.4388 - accuracy: 0.5173 - val_loss: 2.8521 - val_accuracy: 0.1889
Epoch 7/69
53/53 [==============================] - 14s 269ms/step - loss: 1.3811 - accuracy: 0.5454 - val_loss: 3.0588 - val_accuracy: 0.2008

Print model summary

In [ ]:
alexnet.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 100, 100, 96)      34944     
                                                                 
 batch_normalization_16 (Bat  (None, 100, 100, 96)     384       
 chNormalization)                                                
                                                                 
 activation_16 (Activation)  (None, 100, 100, 96)      0         
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 50, 50, 96)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 50, 50, 256)       614656    
                                                                 
 batch_normalization_17 (Bat  (None, 50, 50, 256)     